[Missing Values](https://www.kaggle.com/code/alexisbcook/missing-values)
--- 
Missing values happen. Be prepared for this common challenge in real datasets.
 > 결측치는 발생한다. 실제 데이터에서 흔하게 발생하는 결측치에 대해 대비하자.
 

<br>
이 튜토리얼에서 누락된 값을 다루는 세 가지 접근 방식을 배우게 될 것이며, 이후에 실제 데이터 세트에서 이러한 접근 방식의 효과를 비교할 수 있다.

# 소개
---
데이터가 결측치가 되는데는 여러 가지 방법이 있다. 예를 들어. 
* 2개의 침실이 있는 집은 세 번째 침실의 가치를 포함하지 경우
* 설문 응답자가 수입에 대해 공유하지 않는 경우

대부분의 머신 러닝 라이브러리는(scikit-learn포함) 결측치를 사용해 모델을 구축할 경우 에러를 반환한다. 그러니 우리는 아래 중 한가지 전략을 선택해야 할 것이다. 

# 세 가지 방법

## A Simple Option : Drop columns with Missing Values
가장 간단한 방법은 결측값을 포함한 열을 날려버리는(drop) 것이다. 

![nn](https://i.imgur.com/Sax80za.png)


열의 대다수가 결측값이 아닐 경우에 해당 방식을 사용하면 유용할 수 있는 정보를 포함해 많은 정보를 모델에 액세스 할 수 없게 된다.  
극단적인 예로, 하나의 중요한 열에 단일 항목이 누락된 10,000개의 행의 데이터 집합을 생각해보자. 이 방법을 사용하면 해당 열이 전부 삭제된다.  

## A Better Option : Imputation
**Imputation**은 누락된 값을 숫자로 채우게 된다. 예를 들어, 우리는 각각의 결측치에 대해 평균값을 채우게 된다. 
![nn](https://i.imgur.com/4BpnlPA.png)

대치된 값은(imputate value) 대체로 정확하지 않을진 몰라도, 일반적으로 열을 완전히 삭제해 얻는 모델보다는 더 정확한 모델로 이어진다.

## An Extension To Imputation
대치(Imputation)는 표준적인 접근법으로 일반적인 경우 대부분 잘 작동한다. 그러나, 대치된 값(Imputed values)은 실제 값보다 높거나 낮을 수 있다(혹은 데이터셋에 수집되지 않은 값). 또는 결측값을 포함한 행은 다른 방식으로 고유한 값일 수 있다. 이 경우 모형이 원래 결측값을 고려해 더 나은 예측을 할 수 있다.
![nn](https://i.imgur.com/UWOyg4a.png)

이 접근 방식에서는 이전과같이 결측값을 대치한다. 또한 원 데이터 세트에서 누락된 항목이 있는 각 열에 대해 대치된 항목의 위치를 보여주는 새 열을 추가한다.  

경우에 따라 해당 방식은 유의한 결과 향상을 도모할 수 있다.

# Example
이 예에서 우리는 [Melbourne Housing Dataset](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home)로 작업할 것이다. 우리의 모델은 주택 가격을 예측하기 위해 객실 수와 토지 크기 등의 정보를 사용할 것이다.

우리는 데이터 로딩에 포커스를 맞추치 않는 대신, X_train, X_valid, y_train, y_valid에 이미 훈련 및 검증 데이터가 있는 지점에 있다고 상상할 수 있다.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('./data/melb_data.csv')
# select target
y=data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors=data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid=train_test_split(X, y, train_size=.8, test_size=.2, random_state=0)

## Define Function to Measure Quality of Each Approach
**각 접근 방식의 품질을 측정하기 위한 기능 정의**

우리는 결측치를 다루는 각각 다른 접근 방식을 비교하기 위해 **score_dataset()** 기능을 정의할 것이다. 이 기능은 random forest 모델로 부터 [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error)(MAE)를 보고한다. 

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# 기능(Function) 비교를 위한 다른 접근\
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## Score from Approach1 (Drop Columns with Missing Values)    
* 결과1) 결측치를 포함한 행 제거
---
우리는 train, validtaion set을 모두 가지고 학습하므로, 두 set 모두 같은 열을 삭제하도록 주의해야 한다. 



In [6]:
# 결측치를 포함하는 열 추출
cols_with_missing=[col for col in X_train.columns if X_train[col].isnull().any()]

# training / validation set에서 해당 열 제거
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with)")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with)
183550.22137772635


## Score from Approach2 (Imputation)
* 결과2) 대체하기(Imputation)
---
다음으로, **SimpleImputer**를 사용해 누락된 값을 각 열의 평균 값으로 대체한다.


비록 이는 간단할지라도, 평균값으로 결측치를 채우면 일반적으로 성능이 꽤 좋아진다(이는 데이터별로 다를 수 있다.).   
통계학자들은 귀속값(회귀 대치와 같은)을 결정하는 더 복잡한 방법을 실험했지만, 복잡한 방법은 일반적으로 추가 이점을 제공하지 않는 것을 알 수 있다. 

In [8]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()

imputed_X_train=pd.DataFrame(my_imputer.fit_transform(X_train)) # .fit_transform
imputed_X_valid=pd.DataFrame(my_imputer.transform(X_valid))  # .transform

# 대치된 열 이름 (다시 넣기)
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach2 (Imputation) : ")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach2 (Imputation) : 
178166.46269899711


위에서 보듯이, **행 날리기(Approach1)**보다 **대치(Approach2)**의 MAE 점수가 낮은 것을 알 수 있었으며, 이는 해당 데이터에서 **대치(Approach2)**가 성능이 더 좋다는 것을 의미한다.

## Score from Apporach3 (An Extension to Imputation)
* 결과3) 대상으로의 확장
---
다음으로, 어떤 값이 귀속되었는지 추척하는 동시에 결측값을 귀속한다.

In [30]:
# 원 데이터 변경 방지를 위한 데이터 복사
X_train_plus=X_train.copy()
X_valid_plus=X_valid.copy()

# 대치(=귀속)대상을 나타내는 새로운 열 생성
for col in cols_with_missing:
    X_train_plus[col + '_was_missing']=X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing']=X_valid_plus[col].isnull()
    
# Imputation
my_imputer=SimpleImputer()

imputed_X_train_plus=pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus=pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed colum names; put them back
imputed_X_train_plus.columms = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach3 (An Extension to Imputation)")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

/var/folders/c7/zfhvl_ms5dv3bg_wff_1g0g40000gn/T/ipykernel_10196/3292508454.py:17: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  imputed_X_train_plus.columms = X_train_plus.columns


MAE from Approach3 (An Extension to Imputation)
178927.503183954


/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [29]:
imputed_X_train_plus

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,5.0,3182.0,1.0,1.0,1.0,0.0,153.764119,1940.000000,-37.85984,144.98670,13240.0,0.0,1.0,0.0
1,2.0,8.0,3016.0,2.0,2.0,1.0,193.0,153.764119,1964.839866,-37.85800,144.90050,6380.0,0.0,1.0,1.0
2,3.0,12.6,3020.0,3.0,1.0,1.0,555.0,153.764119,1964.839866,-37.79880,144.82200,3755.0,0.0,1.0,1.0
3,3.0,13.0,3046.0,3.0,1.0,1.0,265.0,153.764119,1995.000000,-37.70830,144.91580,8870.0,0.0,1.0,0.0
4,3.0,13.3,3020.0,3.0,1.0,2.0,673.0,673.000000,1970.000000,-37.76230,144.82720,4217.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,3.0,5.2,3056.0,3.0,1.0,2.0,212.0,153.764119,1964.839866,-37.77695,144.95785,11918.0,0.0,1.0,1.0
10860,3.0,10.5,3081.0,3.0,1.0,1.0,748.0,101.000000,1950.000000,-37.74160,145.04810,2947.0,0.0,0.0,0.0
10861,4.0,6.7,3058.0,4.0,2.0,2.0,441.0,255.000000,2002.000000,-37.73572,144.97256,11204.0,0.0,0.0,0.0
10862,3.0,12.0,3073.0,3.0,1.0,1.0,606.0,153.764119,1964.839866,-37.72057,145.02615,21650.0,0.0,1.0,1.0


위에서 보듯이, **결측치 확장(Approach3)**은 **대치(Approach2)**보다 결과가 살짝 떨어진다.

**어째서 값을 대치하는 것이 행을 삭제하는 것보다 성능이 좋을까?**<br>
training data는 10864행과 12개의 열로 구성되어 있으며 그 중 3가지 열은 결측값을 포함하고 있다. 각각의 열은 절반 미만의 결측치를 포함한다. 열을 제거하는 것은 많은 의미 있는 정보들을 제거하므로, 이보다는 결측값을 대치하는 것이 성능이 더 좋다.

In [32]:
# Shape of traning data (num_rows, num_cols)
print(X_train.shape)
print('\n')
# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column>0])

(10864, 12)


Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


# Conclusion

일반적으로 결측값 대치(Approach2,3)은 결측값이 있는 열을 제거하는 것(Approach1)보다 더 좋은 결과를 산출했다.